In [1]:
import mysql.connector
import os
from collections import defaultdict
from datetime import datetime

mysql_conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="cedur",
        charset="utf8mb4",     # wichtig für Emojis
        use_unicode=True       # stellt sicher, dass Python Unicode-Strings nutzt
    )
cursor = mysql_conn.cursor(dictionary=True)

usergroup_str = "usergroup > 0"



In [2]:
def list_files_and_dirs(directory):
    """Gibt eine Liste von Dateien und Verzeichnissen im angegebenen Verzeichnis zurück."""
    try:
        entries = os.listdir(directory)
        files = [f for f in entries if os.path.isfile(os.path.join(directory, f))]
        dirs = [d for d in entries if os.path.isdir(os.path.join(directory, d))]
        return files, dirs
    except FileNotFoundError:
        print(f"Verzeichnis nicht gefunden: {directory}")
        return [], []
    

def build_fcid_fid_dict(directory, suffix=".csv"):
    """
    Liest ein Verzeichnis ein und erstellt eine verschachtelte Dict-Struktur:
    { fcid: { fid: None } }
    """
    mapping = defaultdict(dict)
    
    for file in os.listdir(directory):
        if file.endswith(suffix):
            try:
                name = os.path.splitext(file)[0]  # "2025090223083472_106900"
                fcid, fid = name.split("_")
                mapping[int(fcid)][int(fid)] = ''   # Platzhalter für späteren Wert
                
                full_path = os.path.join(directory, file)
                # Öffnet die Datei mit dem vollständigen Pfad
                with open(full_path, 'r', encoding='utf-8') as f:
                    # Hier könnten Sie die Zeilen auslesen und speichern
                    for line in f:
                        # print(line.strip().split(";")[3])
                        mapping[int(fcid)][int(fid)] = line.strip().split(";")[3].replace('"','')
            except ValueError:
                continue  # Falls Dateiname nicht passt
    
    return mapping

files, dirs = list_files_and_dirs("C:/xampp/htdocs/cedur/patient_log/")

print("Dateien:", files)
print("Ordner:", dirs)

audit_a = {}  # nur einmal initialisieren, vor der Schleife

for folder in dirs:
    path = "C:/xampp/htdocs/cedur/patient_log/" + folder
    files_sub, dirs_sub = list_files_and_dirs(path)
    
    # Dictionary für den aktuellen Ordner erzeugen
    partial_result = build_fcid_fid_dict(path)

    # Ergebnisse in das Haupt-Result übernehmen
    for fcid, fids in partial_result.items():
        if fcid not in audit_a:
            audit_a[fcid] = {}
        audit_a[fcid].update(fids)  # vorhandene fids erweitern

print(len(audit_a))   # Anzahl der fcids
# for fcid, fids in result.items():
#     print(f"{fcid}: {len(fids)} fids")
       
# def get_one_fcont(fcid, fid):
#     query = f"SELECT fcid, fid, fcont, usergroup FROM `forms_10010` WHERE fcid={fcid} AND fid={fid}"
#     cursor.execute(query)
#     row = cursor.fetchone()
#     if row is None:
#         return "NOT_FOUND"
#     usergroup = row['usergroup']
#     fcont = row["fcont"] if row["fcont"] is not None else "NOT_SET"
#     if usergroup is None:
#         fcont = "DEMO"
#     cursor.fetchall() # zum leeren des Cursors
#     return fcont

def get_one_fcont(fcid, fid):
    usergroup = 0
    # query = "SELECT fcid, fid, fcont, usergroup FROM forms_10010 WHERE fcid=%s AND fid=%s"
    # cursor.execute(query, (fcid, fid))
    
    query = f"SELECT usergroup FROM `forms_10005` WHERE fcid={fcid}"
    cursor.execute(query)
    rows = cursor.fetchall()
    if rows is None:
        return "KEINE VISITE ANGELEGT"
    for row_visite in rows:
        if row_visite['usergroup'] > 0: 
            usergroup = row_visite['usergroup']
    # print("U:",usergroup)
    if usergroup == 0:
        return "DEMO"
    
    query = f"SELECT fcid, fid, fcont, usergroup FROM `forms_10010` WHERE fcid={fcid} AND fid={fid}"
    cursor.execute(query)
    row = cursor.fetchone()
    if row is None:
        return "NOT_FOUND"
    fcont = row["fcont"] if row["fcont"] is not None else "NOT_SET"
    
    # Korrigierte Bedingung: Prüft auf den Wert 0
    
        
    cursor.fetchall()
    return fcont

def get_usergroup(fcid):
    usergroup = 0
    
    query = f"SELECT fcid,fid,fcont FROM `forms_10005` WHERE fcid={fcid} and fid=20"
    cursor.execute(query)
    rows = cursor.fetchall()
    if rows is None:
        usergroup = 0
    else: 
        for row in rows:
            usergroup = row['fcont']
    
    if usergroup == 0 or usergroup==None :
        query = f"SELECT fcid,fid,fcont FROM `forms_10010` WHERE fcid={fcid} and fid=20"
        cursor.execute(query)
        rows = cursor.fetchall()
        if rows is None:
            usergroup = 0
        else: 
            for row in rows:
                usergroup = row['fcont']
    
    return usergroup


print(get_usergroup(2025091611584932))


Dateien: []
Ordner: ['20250914', '20250915', '20250916', '20250917']
72
1


In [ ]:
count_all = 0
count_none = 0

ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

for fcid, fid_a in audit_a.items():
    for fid, fcont_audit in fid_a.items():
        fcont_db = get_one_fcont(fcid, fid)
        if fcont_db == 'NOT_FOUND':
            count_none = count_none + 1
        if fcont_db == 'NOT_FOUND':
            usergroup = get_usergroup(fcid) 
            # print(f"{fcid} {fid}:{fcont_audit} ({usergroup}) => {fcont_db}")
            print(f"INSERT INTO forms_10010 (fcid, fid, fcont, muid, usergroup, mts) VALUES ({fcid}, {fid}, '{fcont_audit}', -{usergroup}, {usergroup}, '{ts}') ON DUPLICATE KEY UPDATE fcont = VALUES(fcont);")
        count_all = count_all + 1

print(count_all,count_none)

# cursor.close()
# mysql_conn.close()

2025091611584932 116700:Meistens (1) => NOT_FOUND
2025091611584932 116800:Ziemlich oft (1) => NOT_FOUND
2025091611584932 116900:Ziemliche Schwierigkeiten (1) => NOT_FOUND
2025091611584932 117000:Ziemlich oft (1) => NOT_FOUND
2025091611584932 117100:Ziemlich oft (1) => NOT_FOUND
2025091611584932 117200:Kein Problem (1) => NOT_FOUND
2025091611584932 117300:Ziemliche Probleme (1) => NOT_FOUND
2025091611584932 117400:Fast Nie (1) => NOT_FOUND
2025091611584932 117500:Ziemlich oft (1) => NOT_FOUND
2025091611584932 117600:Ziemlich oft (1) => NOT_FOUND
2025091611584932 117700:Mäßig (1) => NOT_FOUND
2025091611584932 117800:Mäßig (1) => NOT_FOUND
2025091611584932 117900:Mäßig (1) => NOT_FOUND
2025091611584932 118000:Mäßig (1) => NOT_FOUND
2025091611584932 118100:Ziemlich (1) => NOT_FOUND
2025091611584932 118200:Ziemlich (1) => NOT_FOUND
2025091611584932 118300:Mäßig (1) => NOT_FOUND
2025091611584932 118400:Ein wenig (1) => NOT_FOUND
2025091611584932 118500:Überhaupt nicht (1) => NOT_FOUND
202509